In [1]:
# import libraries needed
from pathlib import Path
import os, yaml, time, sys
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import fft
#from .utils import *
# load in data
file_name = '../../Data/CubicBox_(-250.0, 250.0)_(-250.0, 250.0)_(-250.0, 250.0).npz'
particle_subsample = np.load(file_name)
x, y, z = particle_subsample['x'], particle_subsample['y'], particle_subsample['z']
vx, vy, vz = particle_subsample['vx'], particle_subsample['vy'], particle_subsample['vz'] 
x_range,y_range,z_range=[min(x),max(x)],[min(y),max(y)],[min(z),max(z)]

The below codes are rianna's code to produce linear velocity reconstruction...

In [2]:
# rianna's code
# computes the density field
def compute_density_field(data, grid_size=None, n_grid=None, sample_fraction = 0.021):
    """
    A function that computes the density fluctuation field given the particle subsample for a cubic box

    :param data:        The (x, y, z) coords of the particles in the subsample (N x 3 numpy array)
    :param grid_size:   The size of the 3D grid cells in Mpc (OPTIONAL)
    :param n_grid:      The number of grid cells in 1 dimension such that n_total = n_grid^3 (OPTIONAL)
    """
    if grid_size is None and n_grid is None:
        raise NameError("Either the grid size or the number of grid cells must be specified")
    # compute the box dimensions
    box_lengths = [np.round((np.max(data[:, i]) - np.min(data[:, i]))) for i in range(np.shape(data)[1])]
    if box_lengths.count(box_lengths[0]) != len(box_lengths):
        raise RuntimeError("Input data is not a cubic box. All three side lengths must be equal. Currently (Lx, Ly, "
                           "Lz) = {}".format(box_lengths))
    # set the side length of the grid
    L = int(box_lengths[0])
    # compute the number of grid cells (if the grid size is provided)
    if grid_size is not None:
        ngrid = 500 / grid_size
        if not ngrid.is_integer():
            raise TypeError("Grid spacing is not compatible with the dimension of the simulation. Box size = {} and "
                            "Grid size = {}".format(L, grid_size))
        if n_grid is not None:
            raise NameError(
                "The grid size and the number of cells cannot be specified at the same time. Please specify only one "
                "of these variables.")
    else:
        ngrid = int(n_grid)
        grid_size = L / ngrid
    print('Computing the matter density field, with a grid size of {} Mpc...'.format(grid_size))
    # get an array of the bin edges for the grid 
    edge_array = np.linspace(-L / 2, L/2, int(ngrid + 1))
    # get a tuple of the bin edges for the histogram
    bin_edges = tuple([edge_array for _ in range(np.shape(data)[1])])
    # compute the density in each grid cell 
    density, _ = np.histogramdd(data, bins=bin_edges)
    return density, edge_array

In [3]:
# rianna's code
# computes the density fluctation field
def compute_density_fluctuation_field(data, grid_size=None, n_grid=None, edge_array=None, sample_fraction=0.021):
    """
    A function that computes the density fluctuation field given the particle subsample for a cubic box

    :param data:        Either the (x, y, z) coords of the particles in the subsample (N x 3 numpy array) or the gridded density field (N x N x N numpy array)
    :param grid_size:   The size of the 3D grid cells in Mpc (OPTIONAL)
    :param n_grid:      The number of grid cells in 1 dimension such that n_total = n_grid^3 (OPTIONAL)
    """
    # compute the input data shape 
    data_shape = np.array(data.shape)
    # if the data is a set of (x, y, z) coords compute the matter density field first
    if (data_shape != data_shape[0]).any():
        density_field, edges = compute_density_field(data=data, grid_size=grid_size, n_grid=n_grid, sample_fraction=sample_fraction)
    else: 
        edges = edge_array
        density_field = data
    if grid_size is None:
        grid_size = edges[1] - edges[0]
    print('Computing the matter density fluctuation field...')
    # compute the average particle density in the cubic box
    p_avg = (6912 ** 3) / (2000 ** 3)
    # compute the normalised density field in the cubic box
    p_box = (density_field / sample_fraction) / (grid_size ** len(density_field.shape))
    # compute the matter density fluctuation field
    delta = p_box / p_avg - 1
    return delta, edges

In [4]:
# rianna's code
# called in the function below for the velocity field
def wave_num(kgrid):
    """
    A function that computes the magnitude of the angular wave number k from it's cartesian components on a grid
    :param kgrid: The k-space meshgrid
    """
    kx, ky = kgrid[0], kgrid[1]
    kz = np.zeros_like(kx) if len(kgrid)==2 else kgrid[2]
    return 2 * np.pi * np.sqrt(kx ** 2 + ky ** 2 + kz ** 2)

In [5]:
# rianna's code
# called in the function below for the velocity field
def get_velocity_grids(field, edges):
    """
    A function that returns the grid over which the velocity field will be computed in both real and fourier 
    space. 
    :param field:    The grid corresponding to the matter density field (2D or 3D)
    :param edges:    The side-length of the grid cells used to compute the velocity field.
    """
    # compute the number of dimensions of the density_field 
    N = len(np.shape(field))
    if N not in [2, 3]: 
        raise ValueError("Density field does not appear to be either a 2D or 3D field. The density field has a shape {}".format(np.shape(density_field))) 
    # compute the side length of the density field (in Mpc) 
    L = np.max(edges) - np.min(edges)
    ngrid = len(edges) - 1
    kgrid_arrays = [np.fft.fftfreq(ngrid, L / (ngrid-1)) for _ in range(N-1)]
    kgrid_arrays.extend([np.fft.rfftfreq(ngrid, L / (ngrid-1))])
    # generate the fourier-space velocity grid
    kgrid = np.meshgrid(*reversed(kgrid_arrays)) if N==2 else np.meshgrid(*kgrid_arrays)
    return kgrid

In [6]:
# rianna's code
# computes lin velocity field
def compute_velocity_field(delta, grid_edges, H0=67, omega_m=0.3):
    """
    A function that computes the linear peculiar velocity field across a grid, using a set cosmology
    and a given matter density field.
    :param delta:   The input matter density field (2D or 3D)
    :param grid_edges:    The edges of the bins used to compude the density field (1D array)
    :param H0:              The hubble constant used in the velocity field computation (default is 67
                            km/s/Mpc from Plank2018)
    :param omega_m:         The cosmological matter density parameter used in the velocity field 
                            computation (default is 0.3 from Plank2018)
    """
    print('Computing the linear velocity field...')
    # compute f
    f = omega_m ** 0.6
    # compute the number of dimensions of the density field 
    N = len(np.shape(delta))
    # compute the real space and fourier space grids needed to compute the velocity field.
    k_grid = get_velocity_grids(delta, grid_edges)
    # # compute the wavenumber at each grid point
    k = wave_num(k_grid)
    # compute the k-space density field
    delta_k = fft.rfftn(delta)
    # initialise the fourier space velocity array
    velocity_kx = np.zeros_like(delta_k)
    velocity_ky = np.zeros_like(delta_k)
    velocity_kz = np.zeros_like(delta_k)
    # create the mask for the non-zero wavenumbers
    mask = k != 0
    velocity_kx[mask] = 1j * f * H0 * delta_k[mask] * k_grid[0][mask] / k[mask] ** 2
    velocity_ky[mask] = 1j * f * H0 * delta_k[mask] * k_grid[1][mask] / k[mask] ** 2
    if N == 3: 
        velocity_kz[mask] = 1j * f * H0 * delta_k[mask] * k_grid[2][mask] / k[mask] ** 2
    # compute the inverse transformation to get the real space velocity field
    vx = np.fft.irfftn(velocity_ky, delta.shape)
    vy = np.fft.irfftn(velocity_kx, delta.shape)
    vz = np.fft.irfftn(velocity_kz, delta.shape) if N == 3 else np.zeros_like(vx)
    # return [v_grid[1].T, v_grid[0].T], vx.T, vy.T
    return [vx, vy, vz]

The below codes produce matrices of values...

In [7]:
# split the data into the correct boxes
def full_density_histogram(bins,x=x,y=y,z=z):
    # computes a bins x bins x bins matrix grouping all particles into bins
    # returns the histogram and the edges
    combined=np.column_stack((x,y,z))
    hist,edges=np.histogramdd(combined, bins=[bins,bins,bins])
    return hist, edges

In [8]:
# produce the full 3D overdensity field
def full_overdensity_histogram(bins,x=x):
    # creates a bins x bins x bins matrix with the grouped particles as overdensities
    # returns the matrix and the edges
    grid=500/bins
    fullhist,ed=full_density_histogram(bins)
    smolboxvol=grid**3
    densities=fullhist/smolboxvol
    otherav=len(x)/(500**3)
    overdense=densities/otherav-1
    return overdense, ed

In [9]:
# calculate the mean and std in each box for each component
def full_velos_stds(bins,x=x,y=y,z=z,vx=vx,vy=vy,vz=vz):
    # creates 6 bins x bins x bins matrices with the grouped velocities and stds 
    # returns the 6 matrices
    hist,edges=full_density_histogram(bins)
    x_indices=np.digitize(x,edges[0])-1
    y_indices=np.digitize(y,edges[1])-1
    z_indices=np.digitize(z,edges[2])-1
    xidx = np.clip(x_indices, 0, bins-1)
    yidx = np.clip(y_indices, 0, bins-1)
    zidx = np.clip(z_indices, 0, bins-1)
    directx=np.zeros((bins,bins,bins))
    directy=np.zeros((bins,bins,bins))
    directz=np.zeros((bins,bins,bins))
    stdxsq=np.zeros((bins,bins,bins))
    stdysq=np.zeros((bins,bins,bins))
    stdzsq=np.zeros((bins,bins,bins))
    stdx=np.zeros((bins,bins,bins))
    stdy=np.zeros((bins,bins,bins))
    stdz=np.zeros((bins,bins,bins))
    counts=np.zeros((bins,bins,bins))
    np.add.at(directx, (xidx, yidx, zidx), vx)
    np.add.at(directy, (xidx, yidx, zidx), vy)
    np.add.at(directz, (xidx, yidx, zidx), vz)
    np.add.at(counts, (xidx, yidx, zidx), 1)
    directx[counts > 0] /= counts[counts > 0]
    directy[counts > 0] /= counts[counts > 0]
    directz[counts > 0] /= counts[counts > 0]
    np.add.at(stdxsq, (xidx, yidx, zidx), (vx-directx[xidx,yidx,zidx])**2)
    np.add.at(stdysq, (xidx, yidx, zidx), (vy-directx[xidx,yidx,zidx])**2)
    np.add.at(stdzsq, (xidx, yidx, zidx), (vz-directx[xidx,yidx,zidx])**2)
    stdx[counts > 0] = np.sqrt(stdxsq[counts > 0] / counts[counts>0])
    stdy[counts > 0] = np.sqrt(stdysq[counts > 0] / counts[counts>0])
    stdz[counts > 0] = np.sqrt(stdzsq[counts > 0] / counts[counts>0])
    return directx, directy, directz, stdx, stdy, stdz

In [10]:
# creates the positions to display velocities
def get_positions_velocities(bins,x_range=x_range,y_range=y_range):
    # creates two 1 x bins matrices representing x and y positions
    # returns the two matrices
    xvals=np.array(list(np.linspace(x_range[0]+5,x_range[1]-5,bins))*bins)
    xvals=np.sort(xvals)
    yvals=np.array(list(np.linspace(y_range[0]+5,y_range[1]-5,bins))*bins)
    return xvals, yvals

The below codes produce visualisations...

In [11]:
# produce just a simple density plot
def plot_density(z1,bins,x_range=x_range,y_range=y_range):
    # creates an image of densities in one z slice that is bins thick
    grid=500/bins
    hist,edges=full_density_histogram(bins)
    z_bin = np.where((edges[2][:-1] >= z1) & (edges[2][:-1] < z1+grid))[0]
    slicey=hist[:,:,z_bin[0]]
    plt.imshow(np.rot90(np.log10(slicey+1)), extent=[x_range[0], x_range[1], y_range[0], y_range[1]], cmap='viridis')
    plt.xlabel('x coords')
    plt.ylabel('y coords')
    plt.colorbar(label='$\log_{10}$ number of particles')
    path = '../Figures/NewPlots/Density plot along z=({:.2f},{:.2f}) (gridsize of {:.2f}).png'.format(z1,z1+grid,grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return

<>:11: SyntaxWarning: invalid escape sequence '\l'
<>:11: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_1959/635171461.py:11: SyntaxWarning: invalid escape sequence '\l'
  plt.colorbar(label='$\log_{10}$ number of particles')


In [12]:
# produce just a simple overdensity plot
def plot_overdensity(z1,bins,x_range=x_range,y_range=y_range):
    # creates an image of overdensities in one z slice that is bins thick
    grid=500/bins
    hist,edges=full_overdensity_histogram(bins)
    z_bin = np.where((edges[2][:-1] >= z1) & (edges[2][:-1] < z1+grid))[0]
    slicey=hist[:,:,z_bin[0]]
    plt.imshow(np.rot90(np.log10(slicey+1)), extent=[x_range[0], x_range[1], y_range[0], y_range[1]], cmap='viridis')
    plt.xlabel('x coords')
    plt.ylabel('y coords')
    plt.colorbar(label='$\log_{10}(\delta+1)$')
    path = '../Figures/NewPlots/Overdensity plot along z=({:.2f},{:.2f}) (gridsize of {:.2f}).png'.format(z1,z1+grid,grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return

<>:11: SyntaxWarning: invalid escape sequence '\l'
<>:11: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_1959/488566557.py:11: SyntaxWarning: invalid escape sequence '\l'
  plt.colorbar(label='$\log_{10}(\delta+1)$')


In [13]:
# produce density plot with velocities overlayed
def plot_density_velocities(z1,bins,x_range=x_range,y_range=y_range,vy=vy,vx=vx,x=x,y=y,z=z):
    # creates an image of slice of density field with predicted velocities overlayed
    grid=500/bins
    xve,yve,ex1,ex2,ex3,ex4=full_velos_stds(bins)
    hist,edges=full_density_histogram(bins)
    z_bin = np.where((edges[2][:-1] >= z1) & (edges[2][:-1] < z1+grid))[0]
    slicedense=hist[:,:,z_bin[0]]
    slicex=xve[:,:,z_bin[0]]
    slicey=yve[:,:,z_bin[0]]
    xvals,yvals=get_positions_velocities(bins)
    plt.quiver(xvals,yvals,slicex,slicey,scale_units='xy',angles='xy', color='r',label='x-y velocities (km/s)')
    plt.xlabel('x coords')
    plt.ylabel('y coords')
    plt.legend(loc=2)
    plt.imshow(np.rot90(np.log10(slicedense+1)), extent=[x_range[0], x_range[1], y_range[0], y_range[1]], cmap='viridis')
    plt.colorbar(label='$\log_{10}$ number of particles')
    path = '../Figures/NewPlots/Velocity density plot along z=({:.2f},{:.2f}) (gridsize of {:.2f}).png'.format(z1,z1+grid,grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return    

<>:17: SyntaxWarning: invalid escape sequence '\l'
<>:17: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_1959/818610013.py:17: SyntaxWarning: invalid escape sequence '\l'
  plt.colorbar(label='$\log_{10}$ number of particles')


In [14]:
# produce overdensity plot with velocities overlayed
def plot_overdensity_velocities(z1,bins,x_range=x_range,y_range=y_range,vy=vy,vx=vx,x=x,y=y,z=z):
    # creates an image of slice of overdensity field with predicted velocities overlayed
    grid=500/bins
    xve,yve,ex1,ex2,ex3,ex4=full_velos_stds(bins)
    hist,edges=full_overdensity_histogram(bins)
    z_bin = np.where((edges[2][:-1] >= z1) & (edges[2][:-1] < z1+grid))[0]
    slicedense=hist[:,:,z_bin[0]]
    slicex=xve[:,:,z_bin[0]]
    slicey=yve[:,:,z_bin[0]]
    xvals,yvals=get_positions_velocities(bins)
    plt.quiver(xvals,yvals,slicex,slicey,scale_units='xy',angles='xy', color='r',label='x-y velocities (km/s)')
    plt.xlabel('x coords')
    plt.ylabel('y coords')
    plt.legend(loc=2)
    plt.imshow(np.rot90(np.log10(slicedense+1)), extent=[x_range[0], x_range[1], y_range[0], y_range[1]], cmap='viridis')
    plt.colorbar(label='$\log_{10}(\delta+1)$')
    path = '../Figures/NewPlots/Velocity overdensity plot along z=({:.2f},{:.2f}) (gridsize of {:.2f}).png'.format(z1,z1+grid,grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return    

<>:17: SyntaxWarning: invalid escape sequence '\l'
<>:17: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_1959/1914068332.py:17: SyntaxWarning: invalid escape sequence '\l'
  plt.colorbar(label='$\log_{10}(\delta+1)$')


In [15]:
# produce histogram of overdensities
def histogram_overdensities(bins):
    # creates a histogram of the overdensity matrix
    grid=500/bins
    overdense,ed=full_overdensity_histogram(bins)
    plt.hist(np.log10(overdense.flatten()+1),bins=25)
    plt.xlabel('$\log{10}(\delta+1)$')
    plt.ylabel('Frequency')
    path = '../Figures/NewPlots/Overdensity histogram (gridsize of {:.2f}).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()

<>:7: SyntaxWarning: invalid escape sequence '\l'
<>:7: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_1959/3279838647.py:7: SyntaxWarning: invalid escape sequence '\l'
  plt.xlabel('$\log{10}(\delta+1)$')


In [16]:
# produce histogram of velocities
def histogram_means(bins):
    # creates 4 velocity histograms, 1 for each component, one for the mean
    grid=500/bins
    xve,yve,zve,ex1,ex2,ex3=full_velos_stds(bins)
    plt.hist(xve.flatten(),bins=25)
    plt.xlabel('Velocity x (km/s)')
    plt.ylabel('Frequency')
    path = '../Figures/NewPlots/Velocities x (gridsize of {:.2f}).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    plt.hist(yve.flatten(),bins=25)
    plt.xlabel('Velocity y (km/s)')
    plt.ylabel('Frequency')
    path = '../Figures/NewPlots/Velocities y (gridsize of {:.2f}).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    plt.hist(zve.flatten(),bins=25)
    plt.xlabel('Velocity z (km/s)')
    plt.ylabel('Frequency')
    path = '../Figures/NewPlots/Velocities z (gridsize of {:.2f}).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    means=np.sqrt(xve**2+yve**2+zve**2)
    plt.hist(means.flatten(),bins=25)
    plt.xlabel('Magnitude of velocity (km/s)')
    plt.ylabel('Frequency')
    path = '../Figures/NewPlots/Velocities magnitude (gridsize of {:.2f}).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return

In [17]:
# produce histogram of stds
def histogram_stds(bins):
    # creates 4 std histograms, 1 for each component, one for the magnitude velocity
    grid=500/bins
    ex1,ex2,ex3,stdx,stdy,stdz=full_velos_stds(bins)
    plt.hist(stdx.flatten(),bins=25)
    plt.xlabel('Velocity $\sigma_x$ (km/s)')
    plt.ylabel('Frequency')
    path = '../Figures/NewPlots/Standard deviation in velocities x (gridsize of {:.2f}).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    plt.hist(stdy.flatten(),bins=25)
    plt.xlabel('Velocity $\sigma_y$ (km/s)')
    plt.ylabel('Frequency')
    path = '../Figures/NewPlots/Standard deviation in velocities y (gridsize of {:.2f}).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    plt.hist(stdz.flatten(),bins=25)
    plt.xlabel('Velocity $\sigma_z$ (km/s)')
    plt.ylabel('Frequency')
    path = '../Figures/NewPlots/Standard deviation in velocities z (gridsize of {:.2f}).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    stdt=np.sqrt((stdx**2+stdy**2+stdz**2)/3)
    plt.hist(stdt.flatten(),bins=25)
    plt.xlabel('Velocity $\sigma$ (km/s)')
    plt.ylabel('Frequency')
    path = '../Figures/NewPlots/Standard deviation in magnitude of velocities (gridsize of {:.2f}).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:26: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:26: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_1959/281208505.py:7: SyntaxWarning: invalid escape sequence '\s'
  plt.xlabel('Velocity $\sigma_x$ (km/s)')
/tmp/ipykernel_1959/281208505.py:13: SyntaxWarning: invalid escape sequence '\s'
  plt.xlabel('Velocity $\sigma_y$ (km/s)')
/tmp/ipykernel_1959/281208505.py:19: SyntaxWarning: invalid escape sequence '\s'
  plt.xlabel('Velocity $\sigma_z$ (km/s)')
/tmp/ipykernel_1959/281208505.py:26: SyntaxWarning: invalid escape sequence '\s'
  plt.xlabel('Velocity $\sigma$ (km/s)')


In [18]:
# produce plot of reconstructed velocity field
def plot_slice_reconstruct(xvelo,yvelo,z1,bins,x_range=x_range,y_range=y_range):
    # creates a plot of a slice bins wide of the reconstructed velocity field over the overdensity field
    grid=500/bins
    overdensie,ed=full_overdensity_histogram(bins)
    overdensie+=1
    z_bin = np.where((ed[2][:-1] >= z1) & (ed[2][:-1] < z1+grid))[0]
    slices=overdensie[:,:,z_bin[0]]
    vxslices=xvelo[:,:,z_bin[0]]
    vyslices=yvelo[:,:,z_bin[0]]
    xvals,yvals=get_positions_velocities(bins)
    plt.imshow(np.rot90(np.log10(slices)), extent=[x_range[0], x_range[1], y_range[0], y_range[1]], cmap='viridis')
    plt.colorbar(label='$\log_{10}(\delta+1)$')
    plt.quiver(xvals,yvals,vxslices,vyslices,scale_units='xy',angles='xy', color='r',label='x-y velocities (km/s)')
    plt.legend(loc=2)
    plt.xlabel('x coords')
    plt.ylabel('y coords')
    path = '../Figures/NewPlots/Reconstructed velocities along z=({:.2f},{:.2f}) (gridsize of {:.2f}).png'.format(z1,z1+grid,grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return

<>:13: SyntaxWarning: invalid escape sequence '\l'
<>:13: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_1959/1459265131.py:13: SyntaxWarning: invalid escape sequence '\l'
  plt.colorbar(label='$\log_{10}(\delta+1)$')


In [19]:
# produce a plot of actual vs reconstructed velocity field overlayed on each other
def plot_overlayed_velocities(vvx,vvy,z1,bins,x_range=x_range,y_range=y_range,vy=vy,vx=vx,x=x,y=y,z=z):
    # creates a plot of a slice bins wide of the reconstructed and actual velocity field over the overdensity field
    grid=500/bins
    xve,yve,ex1,ex2,ex3,ex4=full_velos_stds(bins)
    hist,edges=full_overdensity_histogram(bins)
    z_bin = np.where((edges[2][:-1] >= z1) & (edges[2][:-1] < z1+grid))[0]
    slicedense=hist[:,:,z_bin[0]]
    slicex=xve[:,:,z_bin[0]]
    slicey=yve[:,:,z_bin[0]]
    vxslices=vvx[:,:,z_bin[0]]
    vyslices=vvy[:,:,z_bin[0]]
    xvals,yvals=get_positions_velocities(bins)
    plt.imshow(np.rot90(np.log10(slicedense+1)), extent=[x_range[0], x_range[1], y_range[0], y_range[1]], cmap='Greys')
    plt.colorbar(label='$\log_{10}(\delta+1)$')
    plt.quiver(xvals,yvals,slicex,slicey,scale_units='xy',angles='xy', color='r',label='Actual velocities (km/s)')
    plt.quiver(xvals,yvals,vxslices,vyslices,scale_units='xy',angles='xy', color='b',label='Reconstructed velocities (km/s)')
    plt.xlabel('x coords')
    plt.ylabel('y coords')
    plt.legend(loc=2)
    path = '../Figures/NewPlots/Reconstructed vs actual velocities (overlayed) along z=({:.2f},{:.2f}) (gridsize of {:.2f} mpc cubed).png'.format(z1,z1+grid,grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return    

<>:15: SyntaxWarning: invalid escape sequence '\l'
<>:15: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_1959/3964748844.py:15: SyntaxWarning: invalid escape sequence '\l'
  plt.colorbar(label='$\log_{10}(\delta+1)$')


In [20]:
# produce actual vs reconstructed velocities plot
def plot_truevrecon(tvx,tvy,tvz,rvx,rvy,rvz,n,bins):
    # creates a scatter plot of actual (x axis) against reconstructed (y axis)
    # there is a one sigma line in there but its just too close to see
    # takes anything... x, y, z or mean velocities
    realvelo=np.sqrt(tvx**2+tvy**2+tvz**2)
    reconvelo=np.sqrt(rvx**2+rvy**2+rvz**2)
    grid=500/bins
    truey=realvelo.flatten()[::n]
    recon=reconvelo.flatten()[::n]
    plt.scatter(truey,recon,s=0.5)
    plt.xlabel('Actual velocity (km/s)')
    plt.ylabel('Reconstructed velocity (km/s)')
    lit=np.linspace(min(truey),max(truey),10000)
    grad,inter=np.polyfit(truey,recon,1)
    line=grad*lit+inter
    plt.plot(lit,line,color='r')
    texty='Gradient: {:.2f}'.format(grad)
    low,up=plt.ylim()
    lowx,upx=plt.xlim()
    plt.text(lowx+1/10*upx,2*(up-low)/3+low, texty, fontsize=8)
    resid=recon-(grad*truey+inter)
    sigma=np.sqrt(sum(resid)**2/len(resid))
    plt.fill_between(lit,line-sigma,line+sigma,color='r',alpha=0.2)
    path = '../Figures/NewPlots/Reconstructed vs actual magnitude velocities (gridsize of {:.2f} mpc cubed).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return

In [26]:
#produce histogram of means of reconstructed and actual velocities
def histogram_comparison_mag_stds(tvx,tvy,tvz,rvx,rvy,rvz,bins):
    # takes the components of velocity
    # returns std in both samples
    grid=500/bins
    meanreal=np.sqrt(tvx**2+tvy**2+tvz**2)
    meanrecon=np.sqrt(rvx**2+rvy**2+rvz**2)
    plt.hist(meanreal.flatten(),bins=25,color='r',label='Actual')
    plt.hist(meanrecon.flatten(),bins=25,color='b',label='Reconstructed')
    plt.xlabel('Velocity (km/s)')
    plt.ylabel('Frequency')
    plt.legend()
    path = '../Figures/NewPlots/Histogram of reconstructed vs actual magnitude velocities (gridsize of {:.2f} mpc cubed).png'.format(grid)
    plt.savefig(path, dpi=300)
    plt.close()
    return np.std(meanreal), np.std(meanrecon)

Below is the use of each of the above functions...

In [22]:
# every one of the above functions includes the number of grids in 1 dimension, and sometimes a starting z value
gridnum=32 # best for a 2^n number
zstart=0 # some number between -250 and 250 but be careful of grid size obvs

In [118]:
plot_density(zstart,gridnum)

In [119]:
plot_density_velocities(zstart,gridnum)

In [133]:
plot_overdensity(zstart,gridnum)

In [121]:
plot_overdensity_velocities(zstart,gridnum)

In [23]:
overdensity,edges=full_overdensity_histogram(gridnum)

In [24]:
vxcon,vycon,vzcon=compute_velocity_field(overdensity,edges[0])

Computing the linear velocity field...


In [126]:
plot_slice_reconstruct(vxcon,vycon,zstart,gridnum)

In [127]:
plot_overlayed_velocities(vxcon,vycon,zstart,gridnum)

In [128]:
histogram_means(gridnum)

In [129]:
histogram_stds(gridnum)

In [131]:
histogram_overdensities(gridnum)

In [25]:
q,w,e,r,t,y=full_velos_stds(gridnum)

In [240]:
plot_truevrecon(q,w,e,vxcon,vycon,vzcon,1,gridnum)

In [27]:
std1,std2=histogram_comparison_mag_stds(q,w,e,vxcon,vycon,vzcon,gridnum)
print(std1,std2)

178.15481211126217 18.03824635981351
